In [31]:
from vecstack import stacking
from sklearn.metrics import make_scorer

In [33]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import ExtraTreesRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV

In [4]:
import pandas as pd
import numpy as np

## Подготовка данных

Используемые для стекинга модели, такие как RandomForestRegressor, ExtraTreesRegressor принимают на вход лишь числовые переменные и не умеют работать с категориальными, поэтому необходимо сделать преобразование категориальных признаков:

Прочитаем данные:

In [14]:
train = pd.read_csv('train_stack_tax.csv')
test = pd.read_csv('test_stack_tax.csv')

Выделим категориальные признаки:

In [17]:
cat_features_ids = np.where(train.dtypes == object)[0].tolist()
cat_features_ids.append(5)
cat_features_ids.extend([i for i in range(16, 71)])
categorical_features_names = list(train.columns[cat_features_ids])

Объединим тренировочный и тестовый датасеты для dummy-кодирования, добавив столбец price = 0 в тестовую выборку и столбец sample = {0,1} в обе выборки для их разделения в дальнейшем:

In [18]:
train['sample'] = 1
test['sample'] = 0
test['price'] = 0
data = test.append(train, sort=False).reset_index(drop=True)

In [19]:
train.shape, test.shape, data.shape

((18417, 73), (3837, 73), (22254, 73))

Сделаем dummy-кодирование для всех категориальных переменных:

In [20]:
for column in categorical_features_names:
    dummies_train = pd.get_dummies(data[column], prefix=data[column].name)
    data = data.drop(data[column].name, axis=1).join(dummies_train)

In [21]:
data.shape

(22254, 369)

Сделаем разделение на тренировочную и тестовую выборки, а также удалим столбцы sample из обеих выборок и фиктивный столбец price из тестовой: 

In [23]:
train = data[data["sample"] == 1]
test = data[data["sample"] == 0]
train.drop(columns=["sample"], inplace=True)
test.drop(columns=["sample", "price"], inplace=True)

In [24]:
train.shape, test.shape

((18417, 368), (3837, 367))

Сохраним полученные датасеты для дальнейшего использования:

In [25]:
train.to_csv('train_numeric_tax.csv', index=False)
test.to_csv('test_numeric_tax.csv', index=False)

## Подбор гиперпараметров для каждой модели 

В стекинге будем использовать следующие модели: 

1. RandomForestRegressor

2. ExtraTreesRegressor

3. CatBoostRegressor

4. LinearRegression

Первые три модели будут представлять первый уровень, а линейная регрессия будет моделью второго уровня. Нами была рассмотрена также модель KNeighborsRegressor, но её использование в стекинге не дало улучшения. Перед тем как запускать процедуру стекинга, необходимо оптимизировать каждую из моделей, то есть подобрать гиперпараметры. Гиперпараметры для CatBoost были подобраны ранее, для линейной регрессии гиперпараметры подбираться не будут. 

Загрузим данные:

In [29]:
train = pd.read_csv('train_numeric_tax.csv')
test = pd.read_csv('test_numeric_tax.csv')
y = train.price.values
X_train = train.drop(['price'], axis=1)
X_test = test
X_train.shape, X_test.shape, y.shape

((18417, 367), (3837, 367), (18417,))

Проведем подбор параметров для модели RandomForestRegressor. Для этого специальным образом определим функцию скоринга mape, так как её нет среди стандартных функций. Из параметров будем определять количество деревьев, их глубину и нужен ли bootstrap для выборки. Эта часть кода запускалась на Kaggle, так как вычисления ресурсоемки, поэтому просто приведём результаты его работы

In [ ]:
def mape(y_true, y_pred):
    return -np.mean(np.abs((y_pred-y_true)/y_true))


score = make_scorer(mape)

param_grid = [
    {'n_estimators': [100, 500, 1000],
     'max_depth': [10, 50, None], 'bootstrap': [True, False]}
]

grid_search_forest = GridSearchCV(RandomForestRegressor(
    n_jobs=-1), param_grid, cv=3, scoring=score, verbose=2)
grid_search_forest.fit(X_train, y)
grid_search_forest.best_params_

Оптимальные параметры: n_estimators = 1000, max_depth = None, bootstrap = True

In [ ]:
Аналогичную операцию подбора проведём для ExtraTreesRegressor:

In [ ]:
param_grid = [
    {'n_estimators': [100, 500, 1000],
     'max_depth': [10, 50, None], 'bootstrap': [True, False]}
]

grid_search_extra = GridSearchCV(ExtraTreesRegressor(
    n_jobs=-1), param_grid, cv=3, scoring=score, verbose=2)
grid_search_extra.fit(X_train, y)
grid_search_extra.best_params_

Оптимальные параметры: n_estimators = 1000, max_depth = None, bootstrap = True

## Стекинг

Теперь всё готово для проведения стекинга. Воспользуемся для этого библиотекой **stacking**, которая была рекомендована в baseline. Вычисления также проводились на платформе Kaggle. 

In [ ]:
def mape(y_true, y_pred):
    return np.mean(np.abs((y_pred-y_true)/y_true))


# Определим конфигурацию моделей
RANDOM_SEED = 42

lr = LinearRegression(n_jobs=-1)

etc = ExtraTreesRegressor(n_estimators=1000, bootstrap=True, n_jobs=-1,
                          random_state=RANDOM_SEED)
catb = CatBoostRegressor(iterations=3500,
                         learning_rate=0.05,
                         random_seed=RANDOM_SEED,
                         eval_metric='MAPE',
                         verbose=1000
                         )
rf = RandomForestRegressor(random_state=RANDOM_SEED, n_jobs=-1,
                           n_estimators=1000, bootstrap=True)

# Инициализируем модели 1-го уровня
models = [rf, etc, catb]

# Вычислим признаки для передачи в мета-модель
S_train, S_test = stacking(models, X_train, y, X_test,
                           regression=True, metric=mape, n_folds=4,
                           shuffle=True, random_state=RANDOM_SEED, verbose=2)

# Инициализируем модель 2 уровня (мета-модель)
model = lr

# Обучим
model = model.fit(S_train, y)

# Сделаем предсказание и запишем в файл (округление цены до тысяч приводит к незначительному улучшению качества)
y_test_pred = np.exp(model.predict(S_test))
sample_submission = pd.read_csv('sample_submission.csv')
sample_submission['price'] = y_test_pred
sample_submission['price'] = sample_submission['price'].apply(
    lambda x: round(x/1000)*1000)
sample_submission.to_csv('submission_stacking.csv', index=False)
sample_submission.head(10)

### Результаты

Применение стекинга позволило улучшить результат модели до **10.52**, что, несомненно, является улучшением модели. 

## Дополнительное исследование

Суть данного подхода состояла в том, чтобы сделать перестановки 4 используемых моделей так, чтобы каждый раз мета-модель была новой. Получились следующие результаты:

In [10]:
table = pd.DataFrame({'meta_model': [
                      "ExtraTreesRegressor",
                      "RandomForestRegressor",
                      "CatBoostRegressor",
                      "LinearRegressor"],
                      'score': [11.14, 11.20, 11.05, 10.52]})
table

,meta_model,score
0,ExtraTreesRegressor,11.14
1,RandomForestRegressor,11.20
2,CatBoostRegressor,11.05
3,LinearRegressor,10.52


Как видим, все перестановки мета-моделей дали худший результат по сравнению с LinerarRegressor. Попытка осреднить все эти результаты не привела к улучшению, значение метрики после осреднения **10.79**. 

## Финальный результат 

Финальный результат, отображаемый на лидерборде соревнования, равен **10.42**, что соответствует 14 месту из 57. Он получен не совсем честно, так как содержит элемент "читерства" (очень хотелось попробовать так сделать). Отметим, что он оставил нашу команду на том же месте лидерборда. Суть в том, что были взяты лучшие сабмиты и их scores, а дальше произведено осреднение результатов с весами, пропорциональными scores, чего, конечно, в "боевых" задачах сделать нельзя. 